In [ ]:
import sys
import os

from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import torchvision.transforms as transforms

current_dir = os.getcwd()
src_dir = os.path.join(current_dir, '..')

if src_dir not in sys.path:
    sys.path.append(src_dir)

from src.data.classification import TumorClassificationDataset
from src.enums import DataSplit

In [ ]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader

DATASETS_ROOT = os.path.join(current_dir, '..', 'datasets')
BATCH_SIZE = 4

# Define the transformation pipeline for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.ToTensor(),  # Convert images to tensor
    # Add any other transforms here, e.g., normalization
])

# Create an instance of the TumorBinaryClassificationDataset
dataset = TumorClassificationDataset(root_dir=DATASETS_ROOT, split=DataSplit.TRAIN, transform=transform)

# Create a data loader to load the dataset in batches
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

In [ ]:
# Step 1: Prepare the Data
train_dataset = TumorClassificationDataset(root_dir=DATASETS_ROOT, split=DataSplit.TRAIN, transform=transform)
test_dataset = TumorClassificationDataset(root_dir=DATASETS_ROOT, split=DataSplit.TEST, transform=transform)


# Define data loaders for training and testing sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
from src.models.classification.resnet import ResNet, ResidualBlock
import torch.nn as nn
import torch
import torch.nn as nn
import torch.optim as optim
    
# Step 2: Define the Model

class TumorMultiClassifier(nn.Module):
    def __init__(self, num_classes):
        super(TumorMultiClassifier, self).__init__()
        # Load ResNet model
        self.model = ResNet(ResidualBlock, [2, 2, 2, 2], num_classes=num_classes)
        self.model.fc = nn.Linear(512, num_classes)  # Replace the last fully connected layer for multi-class classification

    def forward(self, x):
        return self.model(x)

# Step 3: Define the Loss Function and Optimizer
num_classes = 4  # Number of classes in the tumor dataset
model = TumorMultiClassifier(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 4: Training Loop
# Set the number of training epochs
num_epochs = 10

# Determine the device to use for computation (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the chosen device
model.to(device)

# Start the training loop
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()
    
    # Initialize the running loss for this epoch
    running_loss = 0.0
    
    # Iterate over the batches of images and labels in the training data loader
    for images, labels in train_loader:
        # Move the images and labels to the chosen device
        images, labels = images.to(device), labels.to(device)
        
        # Zero the gradients of the model parameters
        optimizer.zero_grad()
        
        # Forward pass: compute the model outputs for the current batch of images
        outputs = model(images)
        
        # Compute the loss between the model outputs and the actual labels
        loss = criterion(outputs, labels)
        
        # Backward pass: compute the gradients of the loss with respect to the model parameters
        loss.backward()
        
        # Update the model parameters
        optimizer.step()
        
        # Update the running loss
        running_loss += loss.item() * images.size(0)
    
    # Compute the average loss for this epoch
    epoch_loss = running_loss / len(train_dataset)
    
    # Print the epoch number and average loss for this epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# Step 5: Evaluation
# Set the model to evaluation mode. This affects layers like dropout and batchnorm.
model.eval()

# Initialize counters for the total number of images and the number of correctly classified images
correct = 0
total = 0

# Disable gradient computation since we don't need it for evaluation
with torch.no_grad():
    # Iterate over the batches of images and labels in the test data loader
    for images, labels in test_loader:
        # Move the images and labels to the chosen device
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass: compute the model outputs for the current batch of images
        outputs = model(images)
        
        # Get the predicted classes for the current batch of images
        _, predicted = torch.max(outputs.data, 1)
        
        # Update the total number of images
        total += labels.size(0)
        
        # Update the number of correctly classified images
        correct += (predicted == labels).sum().item()

# Compute the accuracy as the percentage of correctly classified images
accuracy = 100 * correct / total

# Print the accuracy on the test set
print(f'Accuracy on the test set: {accuracy:.2f}%')